In [198]:
!pip install -q pyomo

In [199]:
from pyomo.environ import*

$\min\  x_1 - x_2 + x_3\\
\text{ s.t. } x1 ≥ 1,\
 x2 ≥ 1,\ x3 ≥ 2,\\ 2x_1 - x_2 + 2x_3 ≤ 4,\\ 2x_1 - 3x_2 + x_3 ≤ −5 \\
 -x_1 + x_2 - 2x_3 ≤ −1$

In [200]:
model=ConcreteModel()

In [201]:
import numpy as np


In [202]:
N=3
M=3
obj_coeff=np.array([1,-1,1])
constr_coeff=np.array([[2,-1,2],[2,-3,1],[-1,1,-2]])
print('\n',constr_coeff)
col_indices=np.arange(N)
print('\n',col_indices)
row_indices=np.arange(M)
print('\n',row_indices)


 [[ 2 -1  2]
 [ 2 -3  1]
 [-1  1 -2]]

 [0 1 2]

 [0 1 2]


In [203]:
#defining lower and upper bounds to variables
lb=np.array([1,1,2])
ub=np.array([np.inf,np.inf,np.inf])
#defining right hand side term of constraint equation
const_rhs_b=np.array([4,-5,-1])


In [204]:
#defining variables
model.x=Var(col_indices)
model.constraint=ConstraintList()

In [205]:
#defining objective function 
model.objective=Objective(expr=summation(obj_coeff,model.x),sense=minimize)

In [206]:
#defining constraint equation 
for i in row_indices:
  model.constraint.add(sum(constr_coeff[i,j]*model.x[j] for j in col_indices) <=  const_rhs_b[i])

In [207]:
for i in col_indices:
  model.x[i].setlb(lb[i])
  model.x[i].setub(ub[i])

In [ ]:
model.pprint()

#
firstly i am using "glpk solver" 

In [209]:
!apt-get install -y -qq glpk-utils

In [ ]:
opt = SolverFactory('glpk', executable='/usr/bin/glpsol')
result = opt.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

#
now trying to solve with problem using coinor-cbc solver 

In [211]:
!apt-get install -y -qq coinor-cbc

In [212]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

#
 when i am solving above defined problem using glpk solver , i am getting termination condition "other"  which means output of glpk solver is not compatible with pyomo, here output is not meaningful.\
#
but when i am using coinor-cbc solver it is very clearly saying that the termination condition is unbounded, upper bound is infinity.
so we can say there must be some error in defining the problem like sense i.e instead of max we have done min.
here the message is clear so we can try to change the sense of objective function from minimization to maximization to check if the problem is resolved or not.

#2.(a)

In [214]:
model.objective.set_sense(maximize)

In [ ]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

In [ ]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('\nConstraints')

model.constraint.display()

In [ ]:
for i in row_indices:
  if model.constraint[i+1].active==True:
    print(model.constraint[i+1])
    print("active")
  else:
    print(model.constraint[i+1])
    print("inactive")
    

#2.(b)

In [ ]:
model.objective.set_sense(minimize)
model.x[1].setub(5)
model.pprint()

In [ ]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

In [ ]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('\nConstraints')
model.constraint.display()

In [ ]:
for i in row_indices:
  if model.constraint[i+1].active==True:
    print(model.constraint[i+1])
    print("active")
  else:
    print(model.constraint[i+1])
    print("inactive")
    

#2.(C)

In [222]:
model.x[1].setub(np.inf)

In [223]:
#adding one more constraints x2-x3<=6
M=4
row_indices=np.arange(M)
model.constraint.add(expr=model.x[1]-model.x[2]<=6)


In [ ]:
model.pprint()

In [ ]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

In [ ]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('\nConstraints')
model.constraint.display()

In [ ]:
for i in row_indices:
  if model.constraint[i+1].active==True:
    print(model.constraint[i+1])
    print("active")
  else:
    print(model.constraint[i+1])
    print("inactive")
    

#2.(d)

In [228]:
# deactivating objective function and constructing new objective function
model.objective.deactivate()
model.new_objective=Objective(expr=sum(model.x[i] for i in col_indices),sense=minimize)

In [229]:
#deactivating second constraint and adding a different constraint here
model.constraint[2].deactivate()
model.constraint[4].deactivate()
model.constraint.add(expr=model.x[0]+model.x[1]>=25)


In [230]:
M=5
row_indices=np.arange(M)

In [ ]:
model.pprint()

In [ ]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

In [ ]:
# display solution
print('\nObjective = ', model.new_objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('\nConstraints')
model.constraint.display()

In [ ]:
for i in row_indices:
  if model.constraint[i+1].active==True:
    print(model.constraint[i+1])
    print("active")
  else:
    print(model.constraint[i+1])
    print("inactive")
   